<a href="https://colab.research.google.com/github/sahupra1357/Pawpularity/blob/main/PetFinderCNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q kaggle==1.5.8

     |████████████████████████████████| 59 kB 2.3 MB/s 


In [2]:
!ls

sample_data


In [3]:
!mkdir -p ~/.kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pradeepsahu1357","key":"b78a1773d2ce934ad103512a70ef714f"}'}

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
!ls

kaggle.json  sample_data


In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c petfinder-pawpularity-score

 99% 970M/983M [00:07<00:00, 119MB/s]
100% 983M/983M [00:07<00:00, 130MB/s]


In [9]:
!ls

kaggle.json  petfinder-pawpularity-score.zip  sample_data


In [ ]:
!unzip petfinder-pawpularity-score.zip

In [11]:
!ls

kaggle.json			 sample_data		test	  train
petfinder-pawpularity-score.zip  sample_submission.csv	test.csv  train.csv


In [12]:
!ls train |wc -l

9912


In [13]:
!ls test |wc -l

8


In [14]:
import pandas as pd

df_training = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
Ids = df_test["Id"]

df_training["Id"] = df_training["Id"].apply(lambda x :  x + ".jpg")
df_test["Id"] = df_test["Id"].apply(lambda x : x + ".jpg")

In [15]:
df_training.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0.jpg,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db.jpg,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e.jpg,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246.jpg,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2.jpg,0,0,0,1,0,0,1,0,0,0,0,0,72


In [16]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df_training) * TRAIN_PCT)

df_train = df_training[0:TRAIN_CUT]
df_validate = df_training[TRAIN_CUT:]

print(f"Training size: {len(df_train)}")
print(f"Validate size: {len(df_validate)}")

Training size: 8920
Validate size: 992


In [17]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import os

IMAGES_DIR = os.path.abspath("train"+os.sep)

In [18]:
training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory=IMAGES_DIR,
        x_col="Id",
        y_col="Pawpularity",
        target_size=(256, 256),
        batch_size=32,
        class_mode='other')

Found 8920 validated image filenames.


In [19]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate,
        directory=IMAGES_DIR,
        x_col="Id",
        y_col="Pawpularity",
        target_size=(256, 256),
        class_mode='other')

Found 992 validated image filenames.


In [36]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping

lr_schedule = ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=100, decay_rate=0.96,
        staircase=True)

In [37]:

def create_scnn():
  smodel = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(256, 256, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  return smodel



In [26]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (height, width, depth)
	chanDim = -1
		# define the model input
	inputs = Input(shape=inputShape)
	# loop over the number of filters
	for (i, f) in enumerate(filters):
		# if this is the first CONV layer then set the input
		# appropriately
		if i == 0:
			x = inputs
		# CONV => RELU => BN => POOL
		x = Conv2D(f, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
		# flatten the volume, then FC => RELU => BN => DROPOUT
	x = Flatten()(x)
	x = Dense(16)(x)
	x = Activation("relu")(x)
	x = BatchNormalization(axis=chanDim)(x)
	x = Dropout(0.5)(x)
	# apply another FC layer, this one to match the number of nodes
	# coming out of the MLP
	x = Dense(4)(x)
	x = Activation("relu")(x)
	# check to see if the regression node should be added
	if regress:
		x = Dense(1, activation="linear")(x)
	# construct the CNN
	model = Model(inputs, x)
	# return the CNN
	return model

In [38]:
model = create_scnn()

In [27]:
#model = create_cnn(256,256,3, regress=True)

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 127, 127, 64)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 125, 125, 64)      36928     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 246016)            0         
                                                                 
 dense_5 (Dense)             (None, 512)              

In [28]:
#model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 256, 16)      448       
                                                                 
 activation (Activation)     (None, 256, 256, 16)      0         
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 16)     64        
 ormalization)                                                   
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 128, 128, 16)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 32)      4640  

In [40]:
#model.compile(loss = 'mean_squared_error', optimizer='adam')
model.compile(loss = 'mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate = lr_schedule))

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
history = model.fit(train_generator,  
  verbose = 1, 
  validation_data=val_generator, callbacks=[monitor], epochs=25)

Epoch 1/25
279/279 [==============================] - 150s 529ms/step - loss: 571.6473 - val_loss: 422.9357
Epoch 2/25
279/279 [==============================] - 144s 516ms/step - loss: 440.9732 - val_loss: 421.2887
Epoch 3/25
279/279 [==============================] - 144s 516ms/step - loss: 439.4015 - val_loss: 434.8445
Epoch 4/25
279/279 [==============================] - 141s 506ms/step - loss: 430.0449 - val_loss: 420.0814
Epoch 5/25
279/279 [==============================] - 142s 508ms/step - loss: 427.5376 - val_loss: 453.2491
Epoch 6/25
279/279 [==============================] - 146s 523ms/step - loss: 428.3884 - val_loss: 420.4484
Epoch 7/25
279/279 [==============================] - 148s 530ms/step - loss: 424.3523 - val_loss: 411.0642
Epoch 8/25
279/279 [==============================] - 145s 518ms/step - loss: 420.5400 - val_loss: 432.9529
Epoch 9/25
279/279 [==============================] - 138s 493ms/step - loss: 413.7068 - val_loss: 437.8081
Epoch 10/25
279/279 [=======

In [41]:
IMAGES_DIR_TEST = os.path.abspath("test"+os.sep)

In [42]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=IMAGES_DIR_TEST,
        x_col="Id",
        batch_size=1,
        shuffle=False,
        target_size=(256, 256),
        class_mode=None)

Found 8 validated image filenames.


In [43]:
test_generator.reset()
pred = model.predict(test_generator,steps=len(df_test))

In [44]:
df_submit = pd.DataFrame({'Id':Ids,'Pawpularity':pred.flatten()})

In [ ]:
#df_submit.to_csv("submission.csv",index=False)

In [45]:
df_submit

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3.jpg,30.509037
1,43a2262d7738e3d420d453815151079e.jpg,32.279186
2,4e429cead1848a298432a0acad014c9d.jpg,30.909361
3,80bc3ccafcc51b66303c2c263aa38486.jpg,31.737152
4,8f49844c382931444e68dffbe20228f4.jpg,31.148090
5,b03f7041962238a7c9d6537e22f9b017.jpg,32.547138
6,c978013571258ed6d4637f6e8cc9d6a3.jpg,31.725500
7,e0de453c1bffc20c22b072b34b54e50f.jpg,31.450083
